In [1]:
!pip install requests

In [2]:
import requests

In [3]:


url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=z1nYqGtU6IeGZbsoBQFYJjHJ3T3qYAmZd3kGn7BC"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2024-12-21T15:53:48+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"Chile","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio de la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1496","name":"COURT 1","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:62149","name":"Sibur Arena","city_na

In [ ]:
data = response.json()

data['complexes'][3]

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    cv = []
    
    # Loop through complexes
    for cmp in data.get('complexes', []):
        for venue in cmp.get('venues', []):  # Assume venues always exists as a list
            cv.append({
                'complex_id': cmp.get('id'),
                'complex_name': cmp.get('name'),
                'venue_id': venue.get('id'),
                'venue_name': venue.get('name'),
                'city_name': venue.get('city_name'),
                'country_name': venue.get('country_name'),
                'country_code': venue.get('country_code'),
                'timezone': venue.get('timezone')
            })
    
    # Print the result
    for item in cv:
        print(item)

else:
    print(f"Failed to fetch data: {response.status_code} - {response.text}")

In [ ]:
import pandas as pd
cx=pd.DataFrame(cv)
cx

In [1]:
!pip install mysql-connector-python

In [1]:
import mysql.connector

# Establishing the connection to the local MySQL server
connection = mysql.connector.connect(
    host="localhost",         # MySQL server host (localhost for local machine)
    user="root",     # Your MySQL username
    password="Umaneethi@123", # Your MySQL password
    database="sportsradar"  # Database name you want to connect to
)

In [2]:
# Establishing the connection
cursor = connection.cursor()

In [3]:


# Create the tables for complex and venue without primary keys to allow duplicates
cursor.execute('''
CREATE TABLE IF NOT EXISTS Complex_table (
    complex_id VARCHAR(50),
    complex_name VARCHAR(100)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Venue_table (
    venue_id VARCHAR(50),
    venue_name VARCHAR(100),
    city_name VARCHAR(50),
    country_name VARCHAR(100),
    country_code VARCHAR(5),
    timezone VARCHAR(100),
    complex_id VARCHAR(50)
);
''')

In [12]:
# Insert data into complex table
for index, row in cx.iterrows():
    cursor.execute('''
    INSERT INTO Complex_table (complex_id, complex_name)
    VALUES (%s, %s);
    ''', (row['complex_id'], row['complex_name']))

# Insert data into venue table
for index, row in cx.iterrows():
    cursor.execute('''
    INSERT INTO Venue_table (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    ''', (row['venue_id'], row['venue_name'], row['city_name'], row['country_name'], row['country_code'], row['timezone'], row['complex_id']))

# Commit changes and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data has been inserted into MySQL!")


Data has been inserted into MySQL!


In [4]:
!pip install tabulate

In [5]:
from tabulate import tabulate

In [7]:
#1.List all venues along with their associated complex name
cursor.execute(
    '''
    SELECT v.venue_name, c.complex_name
    FROM Venue_table AS v
    INNER JOIN Complex_table AS c ON v.complex_id = c.complex_id Limit 20
    '''
)
results = cursor.fetchall()
headers = ["venue_name", "complex_name"]
table = tabulate(results, headers, tablefmt="Pipe")  # Grid, Box, Pipe, Fancy_grid
print(table)

venue_name      complex_name
--------------  ---------------------
Cancha Central  Nacional
Court One       Estadio de la Cartuja
Centre Court    Estadio de la Cartuja
Court One       Estadio de la Cartuja
Centre Court    Estadio de la Cartuja
TC Dynamo       Sibur Arena
Sibur Arena     Sibur Arena
CENTER COURT    Sibur Arena
COURT 1         Sibur Arena
TC Dynamo       Sibur Arena
Sibur Arena     Sibur Arena
CENTER COURT    Sibur Arena
COURT 1         Sibur Arena
TC Dynamo       Sibur Arena
Sibur Arena     Sibur Arena
CENTER COURT    Sibur Arena
COURT 1         Sibur Arena
TC Dynamo       Sibur Arena
Sibur Arena     Sibur Arena
CENTER COURT    Sibur Arena


In [9]:
#2.Count the number of venues in each complex
cursor.execute(
    '''
    SELECT c.complex_name, 
           (SELECT COUNT(*) FROM Venue_table v WHERE v.complex_id = c.complex_id) AS count_venue
    FROM Complex_table c
    Limit 20
    '''
)
results = cursor.fetchall()
headers = ["complex_name", "count_venue"]
table = tabulate(results, headers, tablefmt="Pipe")  # Grid, Box, Pipe, Fancy_grid
print(table)


complex_name                  count_venue
--------------------------  -------------
Nacional                                1
Estadio de la Cartuja                   2
Estadio de la Cartuja                   2
Sibur Arena                             4
Sibur Arena                             4
Sibur Arena                             4
Sibur Arena                             4
Complexo de Tenis do Jamor              5
Complexo de Tenis do Jamor              5
Complexo de Tenis do Jamor              5
Complexo de Tenis do Jamor              5
Complexo de Tenis do Jamor              5
Ostravar Arena                          6
Ostravar Arena                          6
Ostravar Arena                          6
Ostravar Arena                          6
Ostravar Arena                          6
Ostravar Arena                          6
Varazdin Arena                          8
Varazdin Arena                          8


In [17]:
#3.Get details of venues in a specific country (e.g., Chile)
cursor.execute(
    ''' SELECT * 
        FROM Venue_table 
        WHERE country_name = 'Chile';     
    '''             
)
results = cursor.fetchall()
headers = [" "]
table = tabulate(results, headers, tablefmt="fancy_grid")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

╒════════════════╤═════════════════════════╤══════════════╤═══════╤═════╤══════════════════╤══════════════════╕
│                │                         │              │       │     │                  │                  │
╞════════════════╪═════════════════════════╪══════════════╪═══════╪═════╪══════════════════╪══════════════════╡
│ sr:venue:70045 │ Cancha Central          │ Santiago     │ Chile │ CHL │ America/Santiago │ sr:complex:705   │
├────────────────┼─────────────────────────┼──────────────┼───────┼─────┼──────────────────┼──────────────────┤
│ sr:venue:67857 │ Cancha Central          │ Temuco       │ Chile │ CHL │ America/Santiago │ sr:complex:11769 │
├────────────────┼─────────────────────────┼──────────────┼───────┼─────┼──────────────────┼──────────────────┤
│ sr:venue:67859 │ Cancha 1                │ Temuco       │ Chile │ CHL │ America/Santiago │ sr:complex:11769 │
├────────────────┼─────────────────────────┼──────────────┼───────┼─────┼──────────────────┼────────────

In [18]:
#4.Identify all venues and their timezones
cursor.execute(
    '''  SELECT venue_name as Venues, timezone 
         FROM Venue_table;    
    '''             
)
results = cursor.fetchall()
headers = ["Venues", "timezone"]
table = tabulate(results, headers, tablefmt="pipe")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

| Venues                                  | timezone                       |
|:----------------------------------------|:-------------------------------|
| Cancha Central                          | America/Santiago               |
| Centre Court                            | Europe/Madrid                  |
| Court One                               | Europe/Madrid                  |
| COURT 1                                 | Europe/Moscow                  |
| CENTER COURT                            | Europe/Moscow                  |
| Sibur Arena                             | Europe/Moscow                  |
| TC Dynamo                               | Europe/Moscow                  |
| Central                                 | Europe/Lisbon                  |
| Campo 3                                 | Europe/Lisbon                  |
| Campo 4                                 | Europe/Lisbon                  |
| Campo 1                                 | Europe/Lisbon                  |

In [16]:
#5.Find complexes that have more than one venue
cursor.execute(
    '''
    SELECT complex_name, cnt_venue
    FROM (
        SELECT c.complex_name, COUNT(v.venue_id) AS cnt_venue
        FROM Complex_table c
        JOIN Venue_table v ON c.complex_id = v.complex_id
        GROUP BY c.complex_name
    ) AS venue_counts
    WHERE cnt_venue > 1
    '''
)
results = cursor.fetchall()
headers = ["complex_name", "cnt_venue"]
table = tabulate(results, headers, tablefmt="fancy_grid")  # Grid, Box, Pipe, Fancy_grid
print(table)

╒════════════════════════════════════════════════════════════╤═════════════╕
│ complex_name                                               │   cnt_venue │
╞════════════════════════════════════════════════════════════╪═════════════╡
│ Estadio de la Cartuja                                      │           4 │
├────────────────────────────────────────────────────────────┼─────────────┤
│ Sibur Arena                                                │          16 │
├────────────────────────────────────────────────────────────┼─────────────┤
│ Complexo de Tenis do Jamor                                 │          25 │
├────────────────────────────────────────────────────────────┼─────────────┤
│ Ostravar Arena                                             │          36 │
├────────────────────────────────────────────────────────────┼─────────────┤
│ Varazdin Arena                                             │          64 │
├────────────────────────────────────────────────────────────┼─────────────┤

In [10]:
#6.List venues grouped by country
cursor.execute(
    '''   select venue_name,country_name from Venue_table group by 1,2   
    '''             
)
results = cursor.fetchall()
headers = ["venue_name", "country_name"]
table = tabulate(results, headers, tablefmt="pipe")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

| venue_name                              | country_name         |
|:----------------------------------------|:---------------------|
| Cancha Central                          | Chile                |
| Centre Court                            | Spain                |
| Court One                               | Spain                |
| COURT 1                                 | Russia               |
| CENTER COURT                            | Russia               |
| Sibur Arena                             | Russia               |
| TC Dynamo                               | Russia               |
| Central                                 | Portugal             |
| Campo 3                                 | Portugal             |
| Campo 4                                 | Portugal             |
| Campo 1                                 | Portugal             |
| Campo 8                                 | Portugal             |
| Centre Court                            | Czechia           

In [11]:
#7.Find all venues for a specific complex (e.g., Nacional)
cursor.execute(
    '''
    SELECT v.venue_name, v.venue_id
    FROM Venue_table v
    WHERE v.complex_id = (
        SELECT c.complex_id
        FROM Complex_table c
        WHERE c.complex_name = 'Nacional'
    )
    '''
)
results = cursor.fetchall()
headers = ["venue_name", "venue_id"]
table = tabulate(results, headers, tablefmt="Pipe")  # Grid, Box, Pipe, Fancy_grid
print(table)


venue_name      venue_id
--------------  --------------
Cancha Central  sr:venue:70045
